In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso
from sklearn.metrics import root_mean_squared_error
 
def read_dataframe(path):
    df = pd.read_parquet(path)
    df["duration"] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df[["PULocationID", "DOLocationID"]] = df[["PULocationID", "DOLocationID"]].astype(str)
    return df

df_train = read_dataframe("../data/green_tripdata_2025-01.parquet")
df_val = read_dataframe("../data/green_tripdata_2025-02.parquet")

df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

categorical = ["PU_DO"]
numerical = ["trip_distance"]

dv = DictVectorizer()
X_train = dv.fit_transform(df_train[categorical + numerical].to_dict(orient="records"))
X_val = dv.transform(df_val[categorical + numerical].to_dict(orient="records"))

y_train = df_train["duration"].values
y_val = df_val["duration"].values

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("class-nyc-taxi-experiment")

with mlflow.start_run(run_name="lasso_alpha_0.1"):
    alpha = 0.1
    mlflow.log_param("alpha", alpha)

    model = Lasso(alpha=alpha)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)

    mlflow.log_metric("rmse", rmse)
    mlflow.sklearn.log_model(model, "model")

    print(f"✅ Run finalizado. RMSE = {rmse:.4f}")

2025/10/16 20:33:08 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/16 20:33:08 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/10/16 20:33:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 20:33:22 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/16 20:33:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run finalizado. RMSE = 8.9879


In [3]:
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR
import pickle
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_name="nyc-taxi-model-registry-example")

<Experiment: artifact_location=('file:c:/Users/ivanm/5to Semestre/Proyecto de Ciencia de '
 'Datos/nyc-taxi-predictions-2025/notebooks/mlruns/2'), creation_time=1760498367124, experiment_id='2', last_update_time=1760498367124, lifecycle_stage='active', name='nyc-taxi-model-registry-example', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [4]:
mlflow.sklearn.autolog()

In [5]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

In [6]:
models = [
    
    {"model": GradientBoostingRegressor,
     "params": {"n_estimators": 100, "learning_rate": 0.3, "max_depth": 25, "random_state": 42},
     },
    
    {"model": ExtraTreesRegressor,
     "params": {"n_estimators": 100, "max_depth": 15, "random_state": 42},
     },
    
    {"model": LinearSVR,
     "params": {"C": 1.0, "epsilon": 0}, 
     },

]

In [7]:
with mlflow.start_run(run_name="Nested Runs"):
    for model in models:
        
        model_class = model["model"]
        model_name = model_class.__name__
        params = model["params"]
        
        with mlflow.start_run(run_name=model_name,nested=True):
            
            ml_model = model_class(**params)
           
            ml_model.fit(X_train, y_train)
    
            y_pred = ml_model.predict(X_val)
            
            rmse = root_mean_squared_error(y_val, y_pred)
            mlflow.log_metric("rmse", rmse)
            
            !mkdir models
            with open("models/preprocessor.b", "wb") as f_out:
                pickle.dump(dv, f_out)
                
            mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2025/10/16 20:33:55 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
A subdirectory or file models already exists.
2025/10/16 20:34:19 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
A subdirectory or file models already exists.
c:\Users\ivanm\5to Semestre\Proyecto de Ciencia de Datos\nyc-taxi-predictions-2025\.venv\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
2025/10/16 20:34:31 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
A subdirectory or file models already exists.


### 1er metodo

In [8]:
# Registrar modelo con codigo
from sklearn.ensemble import RandomForestRegressor


with mlflow.start_run(run_name="RandomForestRegressor"):
    ml_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        random_state=42
    )
    
    ml_model.fit(X_train, y_train)
    
    mlflow.sklearn.log_model(
        sk_model=model, 
        artifact_path="model",
        registered_model_name="nyc-taxi-model"
    )
    
    y_pred = ml_model.predict(X_val)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    # !mkdir models
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2025/10/16 20:35:01 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/16 20:35:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 20:35:03 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/10/16 20:35:08 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/16 20:35:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/10/16 20:35:08 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/16 20:35:08 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INF

### 2do Metodo

In [ ]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

### 3er Metodo

In [11]:
from mlflow import MlflowClient

client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

In [ ]:
client.create_registered_model(name="nyc-taxi-model")

In [ ]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = client.create_model_version(
    name="nyc-taxi-model",
    source=run_uri,
    run_id=run_id
)

### Asignar alias y descripciones

In [13]:
### from mlflow import MlflowClient
### estas dos lineas no son necesarias si ya se importo y creo el cliente
### client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

# create "champion" alias for version 1 of model "example-model"
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="champion",
    version=1
)

# set the "challenger" alias to version 2
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger",
    version=5
)

In [17]:
# delete the alias
client.delete_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger"
)

In [21]:
# get a model version by alias
champion_model = client.get_model_version_by_alias(
    name="nyc-taxi-model",
    alias="champion"
)

print(champion_model.params)

None


In [ ]:
# actualizar la descripcion de un modelo con el cliente
client.update_model_version(
    name="nyc-taxi-model",
    version=5,
    description="This model version is a scikit-learn random forest containing 100 decision trees",
)

<ModelVersion: aliases=['challenger'], creation_timestamp=1760668508730, current_stage='None', deployment_job_state=None, description=('This model version is a scikit-learn random forest containing 100 decision '
 'trees'), last_updated_timestamp=1760669219141, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='f3b914dc849943de9183d3cfa9fcc124', run_link=None, source='models:/m-ab517859718d4e9082125b9e5faa19c2', status='READY', status_message=None, tags={}, user_id=None, version=5>

In [22]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
model_version = 1

model_uri = f"models:/{model_name}/{model_version}"

model = mlflow.pyfunc.load_model(
    model_uri=model_uri,
)

model.predict(X_val)

array([ 4.90242314, 24.61007862, 28.282261  , ..., 20.56484041,
       14.11788637, 23.00840632], shape=(44218,))

In [ ]:
#  import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)


array([ 4.90242314, 24.61007862, 28.282261  , ..., 20.56484041,
       14.11788637, 23.00840632], shape=(44218,))

In [ ]:
# desde bash
# Download files using curl
# curl -o ./data/green_tripdata_2025-03.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-03.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: Failed to open the file ./data/green_tripdata_2025-03.parquet: No 

  0 1224k    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (23) client returned ERROR on write of 16384 bytes


### Funciones para comparar modelos 


In [ ]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, alias, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}@{alias}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [27]:
df = read_dataframe("../data/green_tripdata_2025-03.parquet")

In [ ]:
# descargar el preprocesador

run_id = input("Ingrese el run_id")

client.download_artifacts(
    run_id=run_id, 
    path='preprocessor', 
    dst_path='.'
)

'c:\\Users\\ivanm\\5to Semestre\\Proyecto de Ciencia de Datos\\nyc-taxi-predictions-2025\\notebooks\\preprocessor'

In [29]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [30]:
X_test = preprocess(df, dv)

In [31]:
target = "duration"
y_test = df[target].values

In [52]:
# set the "challenger" alias to version 5
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="champion",
    version=1
)

In [50]:
test_model(name="nyc-taxi-model", alias="challenger", X_test=X_test, y_test=y_test)

{'rmse': 741.4639292332431}

In [53]:
test_model(name="nyc-taxi-model", alias="champion", X_test=X_test, y_test=y_test)

{'rmse': 5.986649383018844}